## Bloomberg BQuant Webinar Series: <br> Is there Value in Value?
This is a companion notebook to the "Is there Value in Value?" webinar.

In [ ]:
import bqplot as bqp
import bqviz as bqv
import bql
import numpy as np
import pandas as pd


bq = bql.Service()

In [ ]:
CREDIT_INDEX = 'LUACTRUU Index'
HIGH_YIELD_INDEX = 'LF98TRUU Index'

EQUITY_INDEX = 'RIY Index'
GROWTH_INDEX = 'RLG Index'
VALUE_INDEX = 'RLV Index'

## Value vs junk premium

In [ ]:
def fetch_premium_data(bq):   
    dates = bq.func.range('-1000w', '0w')

    corp_return = _fetch_return(bq, CREDIT_INDEX, dates)
    junk_return = _fetch_return(bq, HIGH_YIELD_INDEX, dates)
    
    junk_prem_frame = junk_return - corp_return
    junk_prem_frame.columns = ['junk_premium']
    
    growth_return = _fetch_return(bq, GROWTH_INDEX, dates)
    value_return = _fetch_return(bq, VALUE_INDEX, dates)
    
    value_prem_frame = value_return - growth_return
    value_prem_frame.columns = ['value_premium']

    frame = pd.concat([value_prem_frame, junk_prem_frame], axis=1)
    return ((frame.dropna() + 1).rolling(500).apply(lambda x: x.prod()) - 1).dropna()


def _fetch_return(bq, security, dates):
    return_ = bq.data.px_last(dates=dates).pct_diff() / 100
    request = bql.Request(security, {'return': return_})
    response = bq.execute(request)
    return response[0].df().reset_index(drop=True).set_index('DATE')
    

prem_frame = fetch_premium_data(bq)
bqv.InteractiveLinePlot(prem_frame, legend='outside').show().set_style()

## Calculating "value of value"

In [ ]:
def _gen_date_grouped_pb_quantiles(bq, dates):
    is_top, is_bottom, dates_vector = _gen_ungrouped_pb_quantiles(bq, dates)
    top = is_top.group(dates_vector).median()
    bottom = is_bottom.group(dates_vector).median()
    return top, bottom


def _gen_ungrouped_pb_quantiles(bq, dates):
    px_to_book = bq.data.px_to_book_ratio(fill='prev', dates=dates)
    dates_vector = px_to_book['AS_OF_DATE']
    quantiles = px_to_book.group(dates_vector).cut(5).ungroup()
    is_top = bq.func.if_(quantiles == 5, px_to_book, np.nan)
    is_bottom = bq.func.if_(quantiles == 1, px_to_book, np.nan)
    return is_top, is_bottom, dates_vector


def fetch_index_value_of_value_data(bq):
    dates = bq.func.range('-50m', '0m')
    top, bottom = _gen_date_grouped_pb_quantiles(bq, dates)

    members = bq.univ.members(EQUITY_INDEX)
    items = {
        'Difference': top - bottom,
        'Ratio': top / bottom}

    request = bql.Request(members, items)
    response = bq.execute(request)

    frames = [x.df() for x in response]
    return pd.concat(frames, axis=0).rename(columns={'AS_OF_DATE': 'date'}).set_index('date', drop=True).loc[:, ['Difference', 'Ratio']]


index_value_frame = fetch_index_value_of_value_data(bq)
bqv.LinePlot(index_value_frame, legend='outside').show()

## Looking for equity sector trends

In [ ]:
def fetch_gics_sector_value_of_value_data(bq, dates):
    
    is_top, is_bottom, dates_vector = _gen_ungrouped_pb_quantiles(bq, dates)
    
    gics_sector = bq.data.gics_sector_name()
    groupers = [dates_vector, gics_sector]

    top = is_top.group(groupers).median()
    bottom = is_bottom.group(groupers).median()
    items = {'value_of_value': top / bottom}

    members = bq.univ.members(EQUITY_INDEX)
    request = bql.Request(members, items)
    response = bq.execute(request)
    
    frame = response[0].df().reset_index().rename(columns={'GICS_SECTOR_NAME()': 'sector'})
    pivot_frame = frame.pivot(index='AS_OF_DATE', columns='sector', values='value_of_value')
    return pivot_frame


dates = bq.func.range('-200w', '0w')

sector_value_frame = fetch_gics_sector_value_of_value_data(bq, dates=dates)
bqv.InteractiveLinePlot(sector_value_frame, legend='outside').show()

## Cross-asset sector trends

In [ ]:
def fetch_gics_sector_junk_premium_data(bq):
    
    junk_yield = _fetch_gics_sector_bond_yield(bq, HIGH_YIELD_INDEX)
    corp_yield = _fetch_gics_sector_bond_yield(bq, CREDIT_INDEX)
    
    junk_yield.rename(columns={'yield': 'junk_yield'}, inplace=True)
    corp_yield.rename(columns={'yield': 'corp_yield'}, inplace=True)

    frame = pd.concat([junk_yield, corp_yield], axis=1)
    frame['junk_premium'] = frame['junk_yield'] - frame['corp_yield']
    
    return frame


def _fetch_gics_sector_bond_yield(bq, index):
    yield_ = bq.data.yield_()
    members = bq.univ.members(index)
    
    gics_sector = bq.func.value(bq.data.gics_sector_name(), bq.univ.issuerof(), mapby='lineage')
    grouped_yield = yield_.group(by=gics_sector).avg()

    request = bql.Request(members, {'yield': grouped_yield})
    response = bq.execute(request)
    
    return response[0].df().drop(['NullGroup'], axis=0).loc[:, ['yield']]


sector_junk_frame = fetch_gics_sector_junk_premium_data(bq)

dataframe = pd.concat(
    [sector_junk_frame.loc[:, 'junk_premium'],
     sector_value_frame.iloc[-1, :].rename('value_premium')],
    axis=1).dropna()


scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

tooltip = bqp.Tooltip(
    fields=['name', 'x', 'y'],
    labels=['Sector', 'Junk premium', 'Value premium'],
    formats=['', '.3f', '.3f'])

mark_scatter = bqp.Scatter(
    x=dataframe['junk_premium'],
    y=dataframe['value_premium'],
    names=dataframe.index,
    tooltip=tooltip,
    display_names=False,
    scales={'x': scale_x, 'y': scale_y})

axis_x = bqp.Axis(scale=scale_x, label='Junk premium')
axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='Value premium')

figure = bqp.Figure(
    marks=[mark_scatter],
    axes=[axis_x, axis_y],
    animation_duration=500,
    title_style={'font-size': '22px'},
    padding_x=0.05,
    padding_y=0.05,
    layout={'width': '100%', 'height': '500px'})

figure